# Multiple Machines

## Architecture

<img src="./helpers/replication-multiple-machines.png" alt="drawing" width="600"/>

## Pre Requisites

- 2 Linux machines with `PostgreSQL` server software and `Patroni`
- 2 Linux machine with `etcd`
- 1 Linux machine with client tools to query postgres

## Setup Network Resolution

This should be done in a plain on-prem environment because in most of other modern environments like Docker Compose, Cloud, K8S, etc... the machines have a DNS-like resolution built in.

Put `IP Name` key values in /etc/hosts.

For Example:

10.252.55.129   pg-1\
10.252.54.125   pg-2\
10.252.54.87    etcd-1\
10.252.54.85    etcd-2

## Configure ETCD Cluster

In [ ]:
#### Inside ETCD Node ####
PEER_PORT=2380
CLIENT_PORT=2379
# For all machines
export ETCD_INITIAL_CLUSTER="etcd-1=http://etcd-1:$PEER_PORT,etcd-2=http://etcd-2:$PEER_PORT" # The initial cluster nodes
export ETCD_INITIAL_CLUSTER_STATE="new"

# For each machine change host
HOST=<host> # Change on other machines
export ETCD_INITIAL_ADVERTISE_PEER_URLS="http://$HOST:$PEER_PORT"
export ETCD_LISTEN_PEER_URLS="http://0.0.0.0:$PEER_PORT"
export ETCD_LISTEN_CLIENT_URLS="http://0.0.0.0:$CLIENT_PORT"
export ETCD_ADVERTISE_CLIENT_URLS="http://$HOST:$CLIENT_PORT"

etcd --data-dir=data/etcd --enable-v2=true --name $HOST

Check out our new ETCD cluster

In [ ]:
#### Inside PG Node ####
# Both Nodes
apt install etcd-client

# First Node
ENDPOINT="etcd-1:2379"
etcdctl --endpoints=$ENDPOINT put hello "world"

# Second Node
ENDPOINT="etcd-2:2379"
etcdctl --endpoints=$ENDPOINT get hello
etcdctl --endpoints=$ENDPOINT del hello

# First Node
etcdctl --endpoints=$ENDPOINT get hello

## Configure Patroni

This should look similar to the single machine setup except:
- etcd machines -> change host etcd-1 / etcd-2 (etcd header)
- pg machines -> change host pg-1 / pg-2 (restapi, pg_hba, postgresql-listen, postgresql-connect_address)

Here is a working config for example of pg-1

```yaml
scope: batman
namespace: /service
name: postgresql0

restapi:
  listen: 0.0.0.0:8008
  connect_address: pg-1:8008

ctl:
 insecure: false # Allow connections to Patroni REST API without verifying certificates

etcd3:
  host: etcd-1:2379

# The bootstrap configuration. Works only when the cluster is not yet initialized.
# If the cluster is already initialized, all changes in the `bootstrap` section are ignored!
bootstrap:
  initdb:  
  - encoding: UTF8
  - data-checksums
  dcs:
    ttl: 30
    loop_wait: 10
    retry_timeout: 10
    maximum_lag_on_failover: 1048576
    primary_start_timeout: 300
#    synchronous_mode: false
    postgresql:
      use_pg_rewind: true
      pg_hba:
      - host replication replicator <hostname pg-1>/32 md5
      - host replication replicator <hostname pg-2>/32 md5
      - host replication replicator 127.0.0.1/32 md5
      - host all all 0.0.0.0/0 md5

postgresql:
  listen: 127.0.0.1,pg-1:5432
  connect_address: pg-1:5432
  data_dir: data/postgresql0
  authentication:
    replication:
      username: replicator
      password: rep-pass
    superuser:
      username: postgres
      password: zalando
    rewind:
      username: rewind_user
      password: rewind_password
# Can be used to make sure more than one Postgres node is not elected to be primary due to Patroni bug / error
# By default on
watchdog: 
  mode: off

tags:
    noloadbalance: false # Only HA
    nostream: false # Change to file based continuous recovery
    # replicatefrom: # Create cascading replication with this
```

In [ ]:
#### Inside PG Node ####
# Both
su postgres
cd ~
patroni <your yaml configuration file>

## Monitor Activity

In [ ]:
curl -s http://localhost:8008/patroni | jq .
curl -s http://localhost:8008/cluster | jq .
curl -s http://localhost:8008/history | jq .